Note:   
This notebook is meant to be open with Google Colab  
(Notebook này được thiết kế cho Google Colab)

In [ ]:
# Required installations
!pip install grpcio==1.63.0
!pip install crewai
!pip install 'crewai[tools]'
!pip install langchain-google-genai==1.0.4
!pip install langchain-anthropic==0.1.20
!pip install langchain_milvus langchain-openai langchain-community
!pip install sentence-transformers


In [ ]:
from crewai import Agent, Task, Crew
from google.colab import userdata
import os

In [ ]:
from langchain_openai import ChatOpenAI

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
model_name = "gpt-3.5-turbo"

open_ai_llm = ChatOpenAI(
    model = model_name,
    temperature = 0.5)


In [ ]:
mathematics_planner = Agent(
    role="Giáo viên Lập Kế Hoạch Nội Dung Giảng dạy",
    goal="Lập kế hoạch các chủ đề hàng tuần hấp dẫn và giáo dục",
    backstory="Bạn chịu trách nhiệm lập kế hoạch các chủ đề hàng tuần.  "
              "Bạn cần đảm bảo nội dung mang tính giáo dục, hấp dẫn và phù hợp với tiêu chuẩn chương trình học. "
              "Kế hoạch của bạn sẽ giúp giáo viên cung cấp các bài học toàn diện và thú vị bao gồm "
              "nhiều chủ đề toán học khác nhau. Mỗi tuần nên xây dựng dựa trên tuần trước, củng cố và mở rộng kiến thức của học sinh, giúp học sinh được rèn năng lực tính toán, năng lực sử dụng ngôn ngữ toán học, năng lực vận dụng"
              "Kĩ thuật dạy học: Kĩ thuật chia nhóm, kĩ thuật đặt câu hỏi, hỏi đáp, động não",
    llm=open_ai_llm,
    allow_delegation=False,
    verbose=True
)
planning_task = Task(
    description=(
        "1. Xác định các chủ đề quan trọng và phù hợp cho {topic} trong vòng {range} và phù hợp với các học sinh {students} "
        "bao gồm các khái niệm cơ bản, định lý, và có thể bao gồm ứng dụng thực tế của {sub_topics}\n"
        "2. Xác định đối tượng học sinh, xem xét "
        "sở thích và những khó khăn của họ trong việc học môn Toán.\n"
        "3. Phát triển một dàn ý chi tiết cho mỗi tuần bao gồm "
        "phần giới thiệu, kỹ thuật dạy học, các điểm chính, hoạt động học tập và câu hỏi thảo luận.\n"
        "4. Bao gồm các từ khóa quan trọng và các nguồn tài liệu tham khảo liên quan."
        "5. Tất cả các output bằng tiếng Việt."
    ),
    expected_output="Một tài liệu kế hoạch nội dung toàn diện "
        "với dàn ý hàng tuần theo tiết học cụ thể, phân tích đối tượng học sinh, "
        "từ khóa quan trọng và các nguồn tài liệu tham khảo.",
    agent=mathematics_planner,
)


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata

os.environ['GOOGLE_API_KEY']=userdata.get('GOOGLE_API_KEY')
model_name = "gemini-1.5-flash"

gemini_llm=ChatGoogleGenerativeAI(model=model_name,
                           verbose=True,
                           temperature=0.5)
lesson_designer = Agent(
    role="Giáo viên Thiết Kế Bài Học Đại Số",
    goal="Thiết kế các bài học chi tiết và chính xác về mặt thông tin cho môn Đại số lớp 9, mỗi tiết học dài 45 phút",
    backstory="Bạn đang làm việc để thiết kế các bài học mới cho môn Đại số 9. "
              "Bạn dựa vào công việc của Giáo viên Toán lớp 9 Lập Kế Hoạch Nội Dung Giảng dạy, người cung cấp dàn ý "
              "và ngữ cảnh liên quan đến chủ đề. "
              "Bạn tuân theo các mục tiêu chính và "
              "hướng dẫn của dàn ý, "
              "do Giáo viên Toán lớp 9 Lập Kế Hoạch Nội Dung Giảng dạy cung cấp. "
              "Bạn cũng cung cấp các thông tin khách quan và không thiên vị "
              "và củng cố chúng bằng các thông tin "
              "được cung cấp bởi Giáo viên Toán lớp 9 Lập Kế Hoạch Nội Dung Giảng dạy. "
              "Bạn đảm bảo bài học của mình có tính tương tác, hấp dẫn và phù hợp với tiêu chuẩn chương trình học.",
    allow_delegation=False,
    llm=gemini_llm,
    verbose=True
)

lesson_plan_task = Task(
    description=(
        "1. Xác định mục tiêu học tập cho tiết học 45 phút về chủ đề {topic}.\n"
        "2. Lập dàn ý chi tiết cho tiết học, bao gồm phần giới thiệu, các điểm chính, hoạt động học tập, và kết luận.\n"
        "3. Phát triển các hoạt động học tập tương tác và hấp dẫn để giúp học sinh hiểu sâu hơn về chủ đề.\n"
        "4. Bao gồm các câu hỏi thảo luận và bài tập để kiểm tra sự hiểu biết của học sinh.\n"
        "5. Đảm bảo nội dung phù hợp với tiêu chuẩn chương trình học và phù hợp với trình độ của học sinh.\n"
        "6. Cung cấp các nguồn tài liệu tham khảo liên quan để hỗ trợ bài học. "
        "7. Tất cả các output bằng tiếng Việt."
    ),
    expected_output="Một kế hoạch bài học chi tiết cho tiết học 45 phút về chủ đề {topic}, "
        "bao gồm mục tiêu học tập, dàn ý chi tiết, hoạt động học tập, câu hỏi thảo luận, bài tập, và các nguồn tài liệu tham khảo.",
    agent=lesson_designer,
)


In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(model_name='keepitreal/vietnamese-sbert')

In [ ]:
from langchain_milvus.vectorstores import Milvus

In [ ]:
from langchain.tools.retriever import create_retriever_tool
from google.colab import userdata
os.environ["PUBLIC_ENDPOINT"] = userdata.get("PUBLIC_ENDPOINT")
os.environ["API_KEY"] = userdata.get("API_KEY")

In [ ]:
retriever = Milvus(
    embedding_function=embedding_model,
    collection_name="s4v_python_oh",
    connection_args={"uri": os.environ.get("PUBLIC_ENDPOINT"),
                     "token": os.environ.get("API_KEY"),
                     "secure": True
    },
).as_retriever(search_params={
        "k":20
    })


retriever_tool = create_retriever_tool(
    retriever=retriever,
    name="LessonRetrieverTool",
    description="Use this tool to retrieve contents from Textbook"
)

In [ ]:
print(retriever_tool.invoke("Đối tượng nghiên cứu của vật lý và mục tiêu của môn vật lý"))

In [ ]:
from langchain_anthropic import ChatAnthropic
from google.colab import userdata

os.environ['ANTHROPIC_API_KEY']=userdata.get('ANTHROPIC_API_KEY')
claude_llm=ChatAnthropic(model="claude-3-opus-20240229", temperature=0, max_tokens=8096)

test_creator = Agent(
    role="Giáo viên tạo Bài Kiểm Tra và Bài Tập Đại Số",
    goal="Tạo các bài kiểm tra và bài tập chính xác và phù hợp cho từng chủ đề của môn Đại số 9 ",
    backstory="Bạn chịu trách nhiệm tạo các bài kiểm tra và bài tập cho môn Đại số 9. "
              "Bạn dựa vào công việc của Giáo viên Lập Kế Hoạch Nội Dung và Giáo viên Thiết Kế Bài Học, "
              "những người cung cấp dàn ý và ngữ cảnh liên quan đến chủ đề. "
              "Bạn tạo các bài kiểm tra và bài tập nhằm đánh giá sự hiểu biết của học sinh "
              "và củng cố kiến thức đã học. "
              "Bạn đảm bảo các bài kiểm tra và bài tập phù hợp với tiêu chuẩn chương trình học "
              "và trình độ của học sinh.",
    allow_delegation=False,
    llm=claude_llm,
    verbose=True,
    tools=[retriever_tool]
)


test_assignment_task = Task(
    description=(
        "1. Xác định các khái niệm và kỹ năng cần kiểm tra cho từng chủ đề {sub_topics}\n"
        "2. Tạo các câu hỏi trắc nghiệm và tự luận để đánh giá sự hiểu biết của học sinh về từng chủ đề trong {sub_topics}.\n"
        "3. Phát triển các bài tập để học sinh thực hành và củng cố kiến thức.\n"
        "4. Bao gồm các câu hỏi và bài tập ở nhiều mức độ khó khác nhau để phù hợp với trình độ của học sinh, đi từ dễ đến khó, bao gồm các bài kiểm tra 15 phút và một tiết (45 phút).\n"
        "5. Đảm bảo các câu hỏi và bài tập phù hợp với tiêu chuẩn chương trình học.\n"
        "6. Cung cấp hướng dẫn chấm điểm và đáp án cho các câu hỏi và bài tập."
        "7. Tất cả các output bằng tiếng Việt."
    ),
    expected_output="Một bộ đề kiểm tra và bài tập chi tiết cho chủ đề {topic} và theo từng {sub_topics}"
        "bao gồm câu hỏi trắc nghiệm, câu hỏi tự luận, bài tập, hướng dẫn chấm điểm và đáp án.",
    agent=test_creator,
)

In [ ]:
crew = Crew(
    # agents=[mathematics_planner, lesson_designer, test_creator],
    agents=[test_creator],
    # tasks=[planning_task,lesson_plan_task, test_assignment_task],
    tasks=[test_assignment_task],
    verbose=2
)


In [ ]:
result = crew.kickoff(inputs={'topic': 'Môn vật lý lớp 9',
                              'range': '6 tuần',
                              'sub_topics': 'căn bậc hai, căn thức bậc hai, hằng đẳng thức, liên hệ giữa phép nhân, phép chia và phép khai phương, biến đổi biểu thức chứa căn thức bậc hai',
                              "students": "30 em học sinh giỏi, 15 em học sinh khá, và 5 em học sinh trung bình"})
print(result)